В данном файле вы найдете простой код Rest Api приложения для интерграции модели. В данном случае авторизация не требуется.

In [ ]:
# импорт библиотек
from flask import Flask, jsonify, make_response, request

import pandas as pd
import pickle
import os.path as path
import sys 

from datetime import datetime
from catboost import CatBoostRegressor 

In [ ]:
# загружаем скалеры, 
# подготовленные данны
# модельку
standard = pickle.load(open('scaler_v2.pkl', 'rb')) 
data_grouped = pd.read_csv('grouped_data.csv')
model = CatBoostRegressor()
model.load_model('delivery_model_v2')

In [ ]:
# 404 если не существует урла
app = Flask(__name__)

@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error':'Not found'}), 404)

In [ ]:
# определяем сезонность
def seazon(x):
    month = x['month']
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    return 4

In [ ]:
# ссылка куда делаем запрос,
# возвращает предсказание
@app.route('/predict', methods=['POST'])
def predict():

    df = pd.DataFrame.from_dict(request.json)    

    df['seazon'] = df.apply(lambda x: seazon(x), axis=1)

    df['avg_time_in_product'] = df.apply(
    lambda x: filter(x, 'days_in_product'), axis=1)

    df['avg_time_in_delivery'] = df.apply(
    lambda x: filter(x, 'days_delivery'), axis=1)

    df = df.drop('seazon', axis=1)
    
    columns = ['distance', 'duration', 'in_stock', 'standart_size', 'client_lat', 'client_lng', 'month', 'day', 'dayofweek', 'avg_time_in_product', 'avg_time_in_delivery', 'avg_diff_time_sm_askona']
    
    df[columns] = standard.transform(df[columns])
    df['preds'] = model.predict(df) 
    
    return jsonify(df['preds'].to_dict())